In [ ]:
!pip install transformers datasets textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 36.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 2

Upload the training and the validation datasets containing the "dialogue" column and the "section_header" column which contains the class of the dialogue.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Training(2).csv to Training(2).csv


In [ ]:
from google.colab import files
upload = files.upload()

Saving Validation.csv to Validation.csv


In [ ]:
import pandas as pd
import io

df=pd.read_csv(io.BytesIO(uploaded['Training(2).csv']))
df2=pd.read_csv(io.BytesIO(upload['Validation.csv']))

In [ ]:
print(type(df["dialogue"][0]))

<class 'str'>


*   The class labels in the dataset are in text form, such as "GENHX","MEDICATIONS","DIAGNOSIS", etc.
Since class labels need to be numerical values for evaluation, we use a label encoder.
*   The validation dataset may contain labels which do not exist in the training
dataset, hence we fit the label encoder on the combined list of labels from both datasets.

In [ ]:
from sklearn.preprocessing import LabelEncoder
combined_labels = pd.concat([df["section_header"],df2["section_header"]])

# Initialize LabelEncoder and fit on combined labels
label_encoder = LabelEncoder()
label_encoder.fit_transform(combined_labels.tolist())

# Transform training labels
df["section_header"] = label_encoder.transform(df["section_header"])

# Transform test labels
df2["section_header"] = label_encoder.transform(df2["section_header"])

*   The "section_header" column after transformation is shown below.



In [ ]:
print(df["section_header"])

0        8
1        8
2        8
3       13
4        2
        ..
1196    16
1197    13
1198     8
1199     7
1200     7
Name: section_header, Length: 1201, dtype: int64


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df)
test_dataset = Dataset.from_pandas(df2)



*   After converting the dataframe into a Huggingface, we preprocess the input text (dialogues).
*   Each dialogue contains the words "Doctor:" and "Patient:". These words do not add value to the class of the dialogue. Hence these words are removed from the strings and replaced with "".
*   The word "also", new line character and carriage return character are also removed, which will leave us with the actual material of the dialogue.



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

words_to_remove = ["Doctor:", "Patient:","also","\n","\r"]

def preprocess_text(text):
    for word in words_to_remove:
        text = text.replace(word, "")
    return text

def tokenize_function(examples):
    # Apply preprocessing
    examples["dialogue"] = [preprocess_text(text) for text in examples["dialogue"]]
    # Tokenize
    return tokenizer(examples["dialogue"], padding="max_length", truncation=True)

print(train_dataset['section_header'])


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("section_header", "labels")
test_dataset = test_dataset.rename_column("section_header", "labels")

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

[8, 8, 8, 13, 2, 15, 15, 0, 7, 7, 8, 8, 13, 15, 8, 16, 8, 2, 8, 13, 15, 14, 15, 0, 1, 1, 2, 0, 8, 7, 2, 7, 2, 8, 19, 7, 8, 4, 8, 15, 8, 6, 7, 17, 8, 7, 16, 7, 6, 7, 16, 4, 8, 7, 7, 7, 7, 19, 16, 8, 16, 7, 16, 16, 16, 7, 7, 6, 13, 13, 7, 2, 3, 7, 15, 15, 7, 7, 2, 8, 7, 15, 15, 15, 15, 13, 7, 2, 5, 11, 7, 7, 7, 5, 7, 13, 8, 2, 15, 7, 15, 7, 8, 1, 0, 1, 7, 7, 12, 13, 8, 5, 8, 15, 8, 15, 19, 8, 16, 8, 7, 15, 15, 15, 19, 15, 3, 0, 7, 7, 15, 8, 6, 1, 0, 7, 7, 15, 7, 19, 8, 8, 0, 7, 7, 8, 1, 15, 8, 7, 16, 4, 7, 7, 0, 13, 15, 8, 0, 7, 7, 8, 7, 15, 15, 8, 16, 7, 0, 3, 7, 7, 8, 13, 2, 1, 3, 7, 2, 2, 0, 15, 8, 2, 8, 19, 8, 8, 19, 7, 8, 7, 7, 8, 8, 13, 7, 8, 7, 8, 7, 13, 1, 15, 1, 6, 0, 7, 8, 8, 15, 16, 7, 15, 19, 3, 7, 2, 7, 10, 7, 7, 2, 8, 3, 7, 8, 15, 15, 18, 7, 7, 0, 8, 5, 8, 7, 7, 13, 8, 7, 8, 7, 7, 8, 13, 8, 8, 17, 15, 15, 6, 2, 19, 15, 19, 7, 8, 2, 2, 4, 0, 13, 15, 2, 13, 17, 15, 8, 7, 8, 7, 16, 15, 5, 3, 7, 13, 8, 13, 7, 8, 15, 2, 19, 16, 8, 15, 4, 15, 7, 7, 7, 2, 8, 3, 8, 8, 7, 0, 8, 2, 7

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]



*   Each dialogue should appear as a string in the dataset and not a list of 1 string.



In [ ]:
print(type(train_dataset['dialogue'][0]))

<class 'str'>


In [ ]:
print((test_dataset['dialogue'][0]))

 When did your pain begin?  I've had low back pain for about eight years now. Is there any injury?   Yeah, it started when I fell in an A B C store. How old are you now? I'm twenty six.   What kind of treatments have you had for this low back pain?  Yeah, I got referred to P T, and I went, but only once or twice, um, and if I remember right, they only did the electrical stimulation, and heat.  I see, how has your pain progressed over the last eight years?  It's been pretty continuous, but it's been at varying degrees, sometimes are better than others.  Do you have any children?  Yes, I had my son in August of two thousand eight, and I've had back pain since giving birth.  Have you had any falls since the initial one?  Yes, I fell four or five days ago while I was mopping the floor.  Did you land on your lower back again? Yes, right onto my tailbone.  Did that make the low back pain worse?  Yes.  Have you seen any other doctors for this issue?  Yes, I saw Doctor X on January tenth two t



*   DistilBERT base model (uncased) is used for training.
*   This model is the distilled version of the BERT base model. The 'uncased' refers to the fact that the model does not distinguish between upper and lower case.
*   This model is a more efficient version of BERT. It retains 97% of BERT's language understanding capabilities while being 60% faster and 40% smaller.

*   The training dataset has 1200 entries while the validation dataset has 100 entries. Hence, training batch size is 32 and validation batch size is 10.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = len(label_encoder.classes_)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=10,
    num_train_epochs=20,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.945661
2,No log,1.480082
3,No log,1.395095
4,No log,1.246480
5,No log,1.170973
6,No log,1.156025
7,No log,1.121633
8,No log,1.228519
9,No log,1.123064
10,No log,1.200796


TrainOutput(global_step=760, training_loss=0.3185504690596932, metrics={'train_runtime': 1091.6846, 'train_samples_per_second': 22.003, 'train_steps_per_second': 0.696, 'total_flos': 3182888309145600.0, 'train_loss': 0.3185504690596932, 'epoch': 20.0})

*   The following evaluates the accuracy, precision, and other metrics for the validation dataset.

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")


Accuracy: 0.72
Precision: 0.6952314814814815
Recall: 0.72
F1 Score: 0.6936521555990818


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




*   The function below performs adversarial attack using methods from the textattack library.
*   WordSwapNeighboringCharacterSwap swaps neighbouring characters within the same word, while WordSwapQWERTY swaps a character with a neighbouring character on the QWERTY keyboard.
*   RepeatModification constraint prevents undoing of character swaps done. The StopwordModification constraint is used to prevent changes from being made to stop words such as "the","as","no","not", etc. in order to preserve the semantic meaning of the text.
* The function takes in text as a parameter and returns the perturbed text according to the methods and constraints and set.



In [ ]:
from textattack.transformations import WordSwapNeighboringCharacterSwap
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import CompositeTransformation

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter

transformation = CompositeTransformation(
    [WordSwapNeighboringCharacterSwap(),WordSwapQWERTY()]
)
# Set up constraints
constraints = [RepeatModification(), StopwordModification()]
# Create augmenter with specified parameters
augmenter = Augmenter(
    transformation=transformation,
    constraints=constraints,
    pct_words_to_swap=0.25,
    transformations_per_example=1,
)

def introduce_spelling_errors(text):
    augmented_text = augmenter.augment(text)

    return augmented_text



*   The dataset in which adversarial examples are to be introduced requires a separate preprocess function since the function needs to introduce errors in the text resulting after removal of words which add no value to the classification.
*  The below function takes in text(string) as a parameter. However, the introduce_spelling_errors function returns the perturbed text as a list consisting of 1 item, the perturbed string. Therefore the preprocess function return s[0] and not s.


In [ ]:
words_to_remove=['Doctor:','Patient:','also','\n','\r']
def preprocess_text2(text):
    for word in words_to_remove:
        text = text.replace(word, "")
    s=introduce_spelling_errors(text)
    return s[0]



*   Import the test dataset and preprocess each dialogue. This will result in an adversarial dataset consisting of grammatical errors.



In [ ]:
from google.colab import files
upload1 = files.upload()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv(io.BytesIO(uploaded['Test 1.csv']))

In [ ]:
df["dialogue"] = [preprocess_text2(text) for text in df["dialogue"]]

* The adversarial dataset is downloaded for reusability.

In [ ]:
import pandas as pd
from google.colab import files
df.to_csv('test1new.csv', index=False)

# Download the file
files.download('test1new.csv')

In [ ]:
from google.colab import files
upload1 = files.upload()

* Import the adversarial dataset.
* The dataset is split for training, validation, and testing.
* The text in the "dialogue" column has already undergone preprocessing. Hence, once imported, we can directly form the tokenized dataset without calling the prepprocess function again.

In [ ]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split

df=pd.read_csv(io.BytesIO(uploaded['test1new.csv']))
df["section_header"] = label_encoder.transform(df["section_header"])
df, df2 = train_test_split(df, test_size=0.20, random_state=42)
df, df3 = train_test_split(df, test_size=0.25, random_state=42)

* Dataframe df is for training and df2 is for testing, while df 3 is for validation (evaluation dataset while training).
* The original adversarial dataset is of size 200. Hence, df has size 120, and df2 and df3 both have size 40.

In [ ]:
print(df3['dialogue'].iloc[0])

 Se are plannong laparoscopic surgery to atke out your gall stones.  Okxy. Will that be covefed by my insurance? Let me ask the stafd. Okay.


In [ ]:
print(type(df['dialogue']))

<class 'pandas.core.series.Series'>


In [ ]:
from datasets import Dataset
test1_dataset = Dataset.from_pandas(df)
test2_dataset = Dataset.from_pandas(df2)
test3_dataset = Dataset.from_pandas(df3)


In [ ]:
print(test1_dataset['dialogue'])

[' Hello, how are you todah? Nto ogod.  What ahppened?  I have a lot of congestion. I  am coughing a lot. Iy efels like I am choking on somtehing.', "Guest_family: My tow and halr year okd osn and five and half year old stepdon are sifk at hkme, they gto a cold, idarrhea and are bomiting.  So, they swa a doctor?Guest_family: eYs, the little noe was taken up to E R and they gave him an inhaler of albuterol.Guest_family_2: Is he getting sick because of me? I was sick with flu soon after delivery. Guest_family: Neptew is sick too. Dir you trvael xnywhere recently?Guest_family: Aruond Christmas tmie we visited da'ds family. They had nidoor cats and dogs.  Anyone smoeks at hkme?Guest_eamily_2: Yse, I do smoke.", " Good afternoon, sif. Mg chart here says that you are a fifty noe yera old white male, is that correct? Good aftrenoon, odctor. Yes, all of that is xorrect.  Grwat, thank you. Plsase, walk me through the history of your symptoms, when did everything begin.  Wdll, I was diagnowed wi

In [ ]:
def tokenize_function2(examples):
    # Tokenize
    return tokenizer(examples["dialogue"], padding="max_length", truncation=True)

In [ ]:
from transformers import AutoTokenizer

test1_dataset = test1_dataset.map(tokenize_function2, batched=True)

test1_dataset = test1_dataset.rename_column("section_header", "labels")

test1_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


test2_dataset = test2_dataset.map(tokenize_function2, batched=True)

test2_dataset = test2_dataset.rename_column("section_header", "labels")

test2_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


test3_dataset = test3_dataset.map(tokenize_function2, batched=True)

test3_dataset = test3_dataset.rename_column("section_header", "labels")

test3_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

* We conduct text attack by performing classification on adversarial input text using our model which has previously only been trained on normal input text.
* Record the performance metrics achieved by the adversarial examples before training.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test2_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test2_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.5
Precision: 0.5
Recall: 0.5
F1 Score: 0.45666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test3_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test3_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.575
Precision: 0.6265625
Recall: 0.575
F1 Score: 0.5536337937424893


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test1_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test1_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.6083333333333333
Precision: 0.6873698646125117
Recall: 0.6083333333333333
F1 Score: 0.6130326518071616


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


* ADVERSARIAL TRAINING OF THE MODEL IS PERFORMED BELOW.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    num_train_epochs=40,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test1_dataset,
    eval_dataset=test3_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.729264
2,No log,4.549721
3,No log,5.087403
4,No log,5.041592
5,No log,5.049173
6,No log,5.231194
7,No log,5.264009
8,No log,5.296408
9,No log,5.299245
10,No log,5.263880


TrainOutput(global_step=1200, training_loss=1.6065896299248076e-07, metrics={'train_runtime': 279.3041, 'train_samples_per_second': 17.186, 'train_steps_per_second': 4.296, 'total_flos': 636047622144000.0, 'train_loss': 1.6065896299248076e-07, 'epoch': 40.0})

* Record the improvement in accuracy of test2_dataset after performing adversarial training.

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test2_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test2_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.65
Precision: 0.5293269230769231
Recall: 0.65
F1 Score: 0.5786972866949984


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


* Earlier, the accuracy of the test2 dataset was 50%. After performing adversarial training, the accuracy is 65%. Therefore, there is a 30% increase in accuracy.
* Similiarly, the Recall and F1 score have also improved by 30% and 26.5%, respectively.

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming trainer is already defined and trained
predictions = trainer.predict(test3_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test3_dataset["labels"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.625
Precision: 0.61875
Recall: 0.625
F1 Score: 0.5964473684210526


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
